In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, random_split
import torch.nn as nn
import numpy as np
from sklearn.metrics import precision_score, f1_score, confusion_matrix

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256,256)),
    transforms.Normalize((0.5,), (0.5),)
])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Test2', transform=transform)

dataset.classes

['Fake', 'Real']

In [ ]:
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64,shuffle=False)

In [ ]:
from torchvision import models

mobilenet = models.mobilenet_v3_small(pretrained=True)
mobilenet

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100%|██████████| 9.83M/9.83M [00:00<00:00, 87.3MB/s]


MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

In [ ]:
mobilenet.classifier[3] = nn.Linear(1024,2) # number of classes in the dataset
mobilenet.classifier

Sequential(
  (0): Linear(in_features=576, out_features=1024, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1024, out_features=2, bias=True)
)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(mobilenet.parameters(), lr=0.001)

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    mobilenet.train()
    running_loss= 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        output = mobilenet(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

mobilenet.eval()
test_labels = []
test_preds = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = mobilenet(images)
        _, preds = torch.max(outputs, 1)
        test_labels.extend(labels.cpu().numpy())
        test_preds.extend(preds.cpu().numpy())

precision = precision_score(test_labels, test_preds, average='binary')
f1 = f1_score(test_labels, test_preds, average='binary')
conf_matrix = confusion_matrix(test_labels, test_preds)

print(f'Precision: {precision}')
print(f'F1 Score: {f1}')
print('Confusion Matrix:')
print(conf_matrix)

mobilenet.save('mobilenet_v3.h5')

Epoch 1/5, Loss: 0.5364394805261067
Epoch 2/5, Loss: 0.21419596725276538
Epoch 3/5, Loss: 0.12681099772453308
Epoch 4/5, Loss: 0.13695711802159036
Epoch 5/5, Loss: 0.10270720667072705
Precision: 0.9076923076923077
F1 Score: 0.6666666666666666
Confusion Matrix:
[[92  6]
 [53 59]]


AttributeError: 'MobileNetV3' object has no attribute 'save'

In [ ]:
for param in mobilenet.parameters():
    param.requires_grad = True

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    mobilenet.train()
    running_loss= 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        output = mobilenet(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

mobilenet.eval()
test_labels = []
test_preds = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = mobilenet(images)
        _, preds = torch.max(outputs, 1)
        test_labels.extend(labels.cpu().numpy())
        test_preds.extend(preds.cpu().numpy())

precision = precision_score(test_labels, test_preds, average='binary')
f1 = f1_score(test_labels, test_preds, average='binary')
conf_matrix = confusion_matrix(test_labels, test_preds)

print(f'Precision: {precision}')
print(f'F1 Score: {f1}')
print('Confusion Matrix:')
print(conf_matrix)

Epoch 1/5, Loss: 0.6118153631687164
Epoch 2/5, Loss: 0.27326498712812153
Epoch 3/5, Loss: 0.1458822406296219
Epoch 4/5, Loss: 0.09852071838187319
Epoch 5/5, Loss: 0.09665674809366465
Precision: 0.7593984962406015
F1 Score: 0.831275720164609
Confusion Matrix:
[[ 68  32]
 [  9 101]]


In [ ]:
x = torch.randn(64, 3, 224, 224)

features_outupt = mobilenet.features(x)
features_outupt.shape

torch.Size([64, 576, 7, 7])

In [ ]:
class ModifiedMobileNet(nn.Module):
    def __init__(self, base_model):
        super(ModifiedMobileNet, self).__init__()
        self.features = base_model.features

        # Add custom layers
        self.custom_layers = nn.Sequential(
            nn.Conv2d(576, 256, kernel_size=(1, 1), stride=(1, 1)),  # Match input channels
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Downsample
            nn.Flatten()
        )

        # Linear layer matching the flattened size
        self.classifier = nn.Linear(256 * 4 * 4, 2)  # Use 4x4 output size from pooling

    def forward(self, x):
        x = self.features(x)
        x = self.custom_layers(x)
        x = self.classifier(x)
        return x


modified_model = ModifiedMobileNet(mobilenet)


In [ ]:
x = torch.randn(64, 3, 224, 224)
output = mobilenet(x)
output.shape

torch.Size([64, 1000])

In [ ]:

# new_features = nn.Sequential(
#     mobilenet.features, # Original features
#     nn.Conv2d(576, 256, kernel_size=(1, 1), stride=(1,1)),
#     nn.ReLU(),
#     nn.MaxPool2d(kernel_size=2, stride=2),
# )


# mobilenet.features = new_features


# mobilenet.classifier[3] = nn.Linear(1024,2)

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    modified_model.train()
    running_loss= 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        output = modified_model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

mobilenet.eval()
test_labels = []
test_preds = []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = modified_model(images)
        _, preds = torch.max(outputs, 1)
        test_labels.extend(labels.cpu().numpy())
        test_preds.extend(preds.cpu().numpy())

precision = precision_score(test_labels, test_preds, average='binary')
f1 = f1_score(test_labels, test_preds, average='binary')
conf_matrix = confusion_matrix(test_labels, test_preds)

print(f'Precision: {precision}')
print(f'F1 Score: {f1}')
print('Confusion Matrix:')
print(conf_matrix)

mobilenet.save('mobilenet_v3.h5')

Epoch 1/5, Loss: 0.737889962536948
Epoch 2/5, Loss: 0.7183711060455867
Epoch 3/5, Loss: 0.7164365010602134
Epoch 4/5, Loss: 0.7224386078970773
Epoch 5/5, Loss: 0.7192525565624237
Precision: 0.5620915032679739
F1 Score: 0.6490566037735849
Confusion Matrix:
[[31 67]
 [26 86]]


AttributeError: 'MobileNetV3' object has no attribute 'save'